In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [5]:
# connect to engine here

In [6]:
engine.connect()

In [7]:
query = """
SELECT 1 number;
"""

pd.read_sql(query, con=engine)

number
0       1

In [32]:
query = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
"""

pd.read_sql(query, con=engine)

schemaname        tablename tableowner tablespace  hasindexes  hasrules  \
0     public  yellow_tax_data       root       None        True     False   

   hastriggers  rowsecurity  
0        False        False

In [19]:
df = pd.read_csv("yellow_tripdata_2021-01.csv", nrows=100)

In [10]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

In [12]:
# create DDL statement to 
print(pd.io.sql.get_schema(df, name="yellow_taxi_data", con=engine))


CREATE TABLE yellow_taxi_data (
	"VendorID" BIGINT, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	"RatecodeID" BIGINT, 
	store_and_fwd_flag TEXT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	congestion_surcharge FLOAT(53)
)




In [20]:
# split into chunks
df_iter = pd.read_csv("yellow_tripdata_2021-01.csv", iterator=True, chunksize=100000)

In [23]:
df = next(df_iter)

In [24]:
len(df)

100000

In [25]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

In [31]:
# create a header for the table name
df.head(n=0).to_sql(name="yellow_tax_data", con=engine, if_exists="replace")

In [28]:
df.head(n=0) # only get the header

Empty DataFrame
Columns: [VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge]
Index: []

In [35]:
query = """
SELECT *
FROM yellow_taxi_data
"""

pd.read_sql(query, con=engine)

Empty DataFrame
Columns: [index, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge]
Index: []

In [36]:
%time df.to_sql(name="yellow_taxi_data", con=engine, if_exists="append")

Wall time: 8.44 s


In [45]:
# counting to table to see if any changes

query = """
SELECT COUNT(1)
FROM yellow_taxi_data
"""

pd.read_sql(query, con=engine)

count
0  1369765

In [38]:
from time import time

In [41]:
# insert the rest of the chunks into the database

while True:
    t_start = time()
    df = next(df_iter)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df.to_sql(name="yellow_taxi_data", con=engine, if_exists="append")
    
    t_end = time()
    
    print("insert another chunk..., took %.3f sec" % (t_end-t_start))

insert another chunk..., took 8.342 sec
insert another chunk..., took 8.385 sec
insert another chunk..., took 8.375 sec
insert another chunk..., took 8.278 sec
insert another chunk..., took 8.671 sec
insert another chunk..., took 8.571 sec
insert another chunk..., took 8.331 sec
insert another chunk..., took 8.587 sec
insert another chunk..., took 8.272 sec
insert another chunk..., took 8.615 sec
insert another chunk..., took 8.542 sec


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


insert another chunk..., took 8.693 sec
insert another chunk..., took 5.509 sec


StopIteration: 

In [44]:
# counting to table to see if any changes

query = """
SELECT MAX(tpep_pickup_datetime), MIN(tpep_pickup_datetime), MAX(total_amount)
FROM yellow_taxi_data
"""

pd.read_sql(query, con=engine)

max                 min      max
0 2021-02-22 16:52:16 2008-12-31 23:05:14  7661.28

# Adding Zones to Postgres

In [12]:
df_zones = pd.read_csv("taxi+_zone_lookup.csv")

In [13]:
df_zones.head()

LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone

In [15]:
# adds data to progres 
df_zones.to_sql(name="zones", con=engine, if_exists="replace")

# Homework 

In [17]:
# looking at data from green taxi trips

df_green = pd.read_csv("green_tripdata_2019-01.csv", nrows=100)

In [18]:
df_green

VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0          2  2018-12-21 15:17:29   2018-12-21 15:18:57                  N   
1          2  2019-01-01 00:10:16   2019-01-01 00:16:32                  N   
2          2  2019-01-01 00:27:11   2019-01-01 00:31:38                  N   
3          2  2019-01-01 00:46:20   2019-01-01 01:04:54                  N   
4          2  2019-01-01 00:19:06   2019-01-01 00:39:43                  N   
..       ...                  ...                   ...                ...   
95         2  2019-01-01 00:15:42   2019-01-01 00:41:04                  N   
96         2  2019-01-01 00:21:28   2019-01-01 00:48:15                  N   
97         2  2019-01-01 00:17:10   2019-01-01 00:23:34                  N   
98         2  2019-01-01 00:25:01   2019-01-01 00:43:55                  N   
99         2  2019-01-01 00:36:56   2019-01-01 00:43:50                  N   

    RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0            1           264           264                5           0.00   
1            1            97            49                2           0.86   
2            1            49           189                2           0.66   
3            1           189            17                2           2.68   
4            1            82           258                1           4.53   
..         ...           ...           ...              ...            ...   
95           1            97           179                1           7.83   
96           1             7           230                1           4.49   
97           1            42            42                1           1.08   
98           1            42           232                2           8.74   
99           1           223             7                5           1.32   

    fare_amount  extra  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0           3.0    0.5      0.5        0.00           0.0        NaN   
1           6.0    0.5      0.5        0.00           0.0        NaN   
2           4.5    0.5      0.5        0.00           0.0        NaN   
3          13.5    0.5      0.5        2.96           0.0        NaN   
4          18.0    0.5      0.5        0.00           0.0        NaN   
..          ...    ...      ...         ...           ...        ...   
95         26.0    0.5      0.5        5.46           0.0        NaN   
96         20.0    0.5      0.5        4.00           0.0        NaN   
97          6.5    0.5      0.5        0.00           0.0        NaN   
98         25.0    0.5      0.5        0.00           0.0        NaN   
99          7.0    0.5      0.5        2.00           0.0        NaN   

    improvement_surcharge  total_amount  payment_type  trip_type  \
0                     0.3          4.30             2          1   
1                     0.3          7.30             2          1   
2                     0.3          5.80             1          1   
3                     0.3         19.71             1          1   
4                     0.3         19.30             2          1   
..                    ...           ...           ...        ...   
95                    0.3         32.76             1          1   
96                    0.3         25.30             1          1   
97                    0.3          7.80             2          1   
98                    0.3         26.30             2          1   
99                    0.3         10.30             1          1   

    congestion_surcharge  
0                    NaN  
1                    NaN  
2                    NaN  
3                    NaN  
4                    NaN  
..                   ...  
95                   NaN  
96                   NaN  
97                   NaN  
98                   NaN  
99                   NaN  

[100 rows x 20 columns]

In [19]:
print(pd.io.sql.get_schema(df_green, name="green_taxi_data", con=engine))


CREATE TABLE green_taxi_data (
	"VendorID" BIGINT, 
	lpep_pickup_datetime TEXT, 
	lpep_dropoff_datetime TEXT, 
	store_and_fwd_flag TEXT, 
	"RatecodeID" BIGINT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	ehail_fee FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	payment_type BIGINT, 
	trip_type BIGINT, 
	congestion_surcharge FLOAT(53)
)


